# Notes for stress data
- StudentGrades.txt contains the grades for each student 
- The Data.zip file contains folders for each participants named as S1, S2, etc.
- Under each of the folders corresponding to each partcipants, there are three folders 'Final', 'Midterm 1', and 'Midterm 2', corresponding to three exams.
- Each of the folders contains csv files: 'ACC.csv', 'BVP.csv', 'EDA.csv', 'HR.csv', 'IBI.csv', 'tags.csv', 'TEMP.csv', and 'info.txt'.
- 'info.txt' contains detailed information of each of these files.
- All the unix time stamps are date shifted for deidentification but not time shifted. The date shift have been carried out such a way that it does not change the status of the day light saving settings (CT/CDT) of a day.
- All exam starts at 9:00 AM (CT or CDT depending on the date corresponding to the unix time stamp). Mid terms are 1.5 hr long and final is 3 hr long.
- Sampling frequency of the arrays are given with the structs
- The dataset contains two female and eight male participants, however the gender is not mentioned for the purpose of deidentification

The first row is the initial time of the session expressed as unix timestamp in UTC.
The second row is the sample rate expressed in Hz.

TEMP.csv
Data from temperature sensor expressed degrees on the Celsius (°C) scale.

EDA.csv
Data from the electrodermal activity sensor expressed as microsiemens (μS).

BVP.csv
Data from photoplethysmograph.

ACC.csv
Data from 3-axis accelerometer sensor. The accelerometer is configured to measure acceleration in the range [-2g, 2g]. Therefore the unit in this file is 1/64g.
Data from x, y, and z axis are respectively in first, second, and third column.

IBI.csv
Time between individuals heart beats extracted from the BVP signal.
No sample rate is needed for this file.
The first column is the time (respect to the initial time) of the detected inter-beat interval expressed in seconds (s).
The second column is the duration in seconds (s) of the detected inter-beat interval (i.e., the distance in seconds from the previous beat).

HR.csv
Average heart rate extracted from the BVP signal.The first row is the initial time of the session expressed as unix timestamp in UTC.
The second row is the sample rate expressed in Hz.


tags.csv
Event mark times.
Each row corresponds to a physical button press on the device; the same time as the status LED is first illuminated.
The time is expressed as a unix timestamp in UTC and it is synchronized with initial time of the session indicated in the related data files from the corresponding session.

In [1]:
import os
from zipfile import ZipFile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [23]:
main_file_name = 'a-wearable-exam-stress-dataset-for-predicting-cognitive-performance-in-real-world-settings-1.0.0.zip'
sub_file_name = 'Data.zip'

# New dir
path = main_file_name[:-4]

full_path = os.path.join(path, sub_file_name)

In [24]:
def zip_extraction(file_name: str, extract_dir: str = None) -> None:
    with ZipFile(file_name, 'r') as zip:
        zip.printdir()
        print('Extrayendo archivos')
        zip.extractall(path=extract_dir)
        print('Finalizado')

In [25]:
zip_extraction(main_file_name) # Extracting main zip
zip_extraction(full_path, path) # Extracting zip with students data contained. 

File Name                                             Modified             Size
a-wearable-exam-stress-dataset-for-predicting-cognitive-performance-in-real-world-settings-1.0.0/Data.zip 2022-04-03 21:13:28     86016326
a-wearable-exam-stress-dataset-for-predicting-cognitive-performance-in-real-world-settings-1.0.0/LICENSE.txt 2022-05-24 08:04:06        20402
a-wearable-exam-stress-dataset-for-predicting-cognitive-performance-in-real-world-settings-1.0.0/SHA256SUMS.txt 2022-05-26 12:50:46          309
a-wearable-exam-stress-dataset-for-predicting-cognitive-performance-in-real-world-settings-1.0.0/StudentGrades.txt 2021-12-14 12:49:18          470
a-wearable-exam-stress-dataset-for-predicting-cognitive-performance-in-real-world-settings-1.0.0/readme.md 2022-04-03 21:56:08         1104
Extrayendo archivos
Finalizado
File Name                                             Modified             Size
Data/S1/                                       2022-04-03 19:58:54            0
Data/S1/Final/ 

Finalizado


## Reading files

In [141]:
# Define column names for each variable
physio_variables = {'ACC': ['x', 'y', 'z'], 
                    'BVP': ['BVP'], 
                    'EDA': ['EDA'], 
                    'HR': ['HR'], 
                    'IBI':['Time', 'Interval duration'], 
                    'TEMP':['TEMP']
                   }

# Define types of exam
exam_list = ['Final', 'Midterm 1', 'Midterm 2']

# Define students list
students_list = [f'S{i}' for i in range(1, 11)]

In [148]:
students_dict = {}

for student in students_list:
    students_dict[student] = {}
    for exam in exam_list:
        students_dict[student][exam] = {}
        for key, values in physio_variables.items():
            read_data = pd.read_csv(f'a-wearable-exam-stress-dataset-for-predicting-cognitive-performance-'\
                                    f'in-real-world-settings-1.0.0/Data/{student}/{exam}/{key}.csv', names=values)
            students_dict[student][exam][key] = read_data

In [152]:
students_dict['S2']['Final']['HR'].head(10)

,HR
0,1.544027e+09
1,1.000000e+00
2,1.160000e+02
3,8.250000e+01
4,9.467000e+01
5,9.150000e+01
6,8.820000e+01
7,8.483000e+01
8,8.443000e+01
9,8.400000e+01
